In [61]:
using SFRM
using P4P
using CrystalInfoFramework
using Crystalline
using DataFrames
using Unitful
using UnitfulEquivalences
using Interpolations
using StaticArrays
using JamesCore
using XLSX
using GeometryBasics

In [62]:
p4p = P4P.load("D:/p4p/20250615_YEu_AM-17-22.p4p")

Dict{String, Any} with 21 entries:
  "SITEID" => (nothing, nothing)
  "TITLE"  => "CCU_TSuhih"
  "ZEROS"  => (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
  "MOSAIC" => (0.47, 1.88)
  "REF"    => Any[(1024, false, false, false, true, 4, 0, 2, 28.0, 19.312, 0.0,…
  "CELL"   => (10.5892, 10.5892, 10.5892, 90.0, 90.0, 90.0, 1187.36)
  "DNSMET" => "? "
  "CHEM"   => "? "
  "LIMITS" => (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
  "ADPAR"  => (387.22, 503.932, 28.993, 1024, 768)
  "ORT"    => [-0.0109463 -0.0919879 0.0183471; 0.0650277 -0.0207543 -0.06526; …
  "CCOLOR" => "? "
  "FILEID" => ("APEX3", v"2019.1.0-0", v"4.0.0", DateTime("2025-06-15T09:56:45"…
  "CSIZE"  => (nothing, nothing, nothing, nothing, 25.85)
  "SOURCE" => ("CU", 1.54184, 1.54056, 1.54439, 2.0, 50.0, 1.1)
  "MORPH"  => "? "
  "SMAP"   => (1,)
  "ADCOR"  => (0.6017, 2.0131, -0.036, -0.1601, 0.34, 0.0081)
  "BRAVAI" => ('c', 'I')
  "DATA"   => Any[("SPATIAL", "linear", "linear", "3.0", 384.0, 512.0, 17.0, 31…
  "CELLSD" => (0.000

In [63]:
# cif = first(Cif("D:/cif/BGO/mo_BGO_0m.cif"))[2]
cif = first(Cif("D:/cif/Y2O3/Y2O3_CollCode237970.cif"))[2]
group = cif;

In [64]:
sg = spacegroup(parse(Int, cif["_space_group_IT_number"][1]))

SpaceGroup{3} ⋕206 (Ia-3) with 48 operations:
 1
 {2₀₀₁|½,0,½}
 {2₀₁₀|0,½,½}
 {2₁₀₀|½,½,0}
 3₁₁₁⁺
 {3₋₁₁₋₁⁺|½,½,0}
 {3₋₁₁₁⁻|½,0,½}
 {3₋₁₋₁₁⁺|0,½,½}
 3₁₁₁⁻
 {3₋₁₁₁⁺|0,½,½}
 {3₋₁₋₁₁⁻|½,½,0}
 {3₋₁₁₋₁⁻|½,0,½}
 -1
 {m₀₀₁|½,0,½}
 {m₀₁₀|0,½,½}
 {m₁₀₀|½,½,0}
 -3₁₁₁⁺
 {-3₋₁₁₋₁⁺|½,½,0}
 {-3₋₁₁₁⁻|½,0,½}
 {-3₋₁₋₁₁⁺|0,½,½}
 -3₁₁₁⁻
 {-3₋₁₁₁⁺|0,½,½}
 {-3₋₁₋₁₁⁻|½,½,0}
 {-3₋₁₁₋₁⁻|½,0,½}
 {1|½,½,½}
 {2₀₀₁|0,½,0}
 {2₀₁₀|½,0,0}
 {2₁₀₀|0,0,½}
 {3₁₁₁⁺|½,½,½}
 {3₋₁₁₋₁⁺|0,0,½}
 {3₋₁₁₁⁻|0,½,0}
 {3₋₁₋₁₁⁺|½,0,0}
 {3₁₁₁⁻|½,½,½}
 {3₋₁₁₁⁺|½,0,0}
 {3₋₁₋₁₁⁻|0,0,½}
 {3₋₁₁₋₁⁻|0,½,0}
 {-1|½,½,½}
 {m₀₀₁|0,½,0}
 {m₀₁₀|½,0,0}
 {m₁₀₀|0,0,½}
 {-3₁₁₁⁺|½,½,½}
 {-3₋₁₁₋₁⁺|0,0,½}
 {-3₋₁₁₁⁻|0,½,0}
 {-3₋₁₋₁₁⁺|½,0,0}
 {-3₁₁₁⁻|½,½,½}
 {-3₋₁₁₁⁺|½,0,0}
 {-3₋₁₋₁₁⁻|0,0,½}
 {-3₋₁₁₋₁⁻|0,½,0}

In [65]:
pg = pointgroup(sg);

In [94]:
function parse_cif_coords(str::AbstractString)
    m = match(r"(.*)\(\d+\)", str)
    num = isnothing(m) ? str : m[1]
    parse(Float64, num)
end
atomcoords = get_loop(cif, "_atom_site_label")[:, [:_atom_site_type_symbol, :_atom_site_fract_x, :_atom_site_fract_y, :_atom_site_fract_z]]
select!(atomcoords, :_atom_site_type_symbol => ByRow(x -> String(x)) => :atom, :_atom_site_fract_x => ByRow(parse_cif_coords) => :x, :_atom_site_fract_y => ByRow(parse_cif_coords) => :y, :_atom_site_fract_z => ByRow(parse_cif_coords) => :z)

Row,atom,x,y,z
,String,Float64,Float64,Float64
1,Y3+,0.25,0.25,0.25
2,Y3+,-0.03241,0.0,0.25
3,O2-,0.3908,0.1518,0.3803


In [95]:
transform!(atomcoords, [:x, :y, :z] => ByRow((x,y,z) -> 1 .- (x,y,z)) => [:x, :y, :z], renamecols=false)

Row,atom,x,y,z
,String,Float64,Float64,Float64
1,Y3+,0.75,0.75,0.75
2,Y3+,1.03241,1.0,0.75
3,O2-,0.6092,0.8482,0.6197


In [96]:
λ = 1.5405929u"Å"
E = ustrip(uconvert(u"keV", λ, Spectral()))

8.047823564109652

In [97]:
function nist_form_factor(data::AbstractMatrix; E = E)
    itp = linear_interpolation(data[:, 1], data[:, 2])
    itp(E)
end
form_factors = DataFrame(atom = ["O", "Ge", "Bi", "Y3+", "O2-"],
    f = [
        nist_form_factor([7.902609E+00  8.05336E+00; 8.447890E+00  8.04808E+00]),
        nist_form_factor([7.902609E+00  3.10798E+01; 8.447890E+00  3.09116E+01]),
        nist_form_factor([8.0438952E+00  7.99123E+01; 8.0841147E+00  7.99025E+01]),
        nist_form_factor([7.902609E+00  3.56076E+01; 8.447890E+00  3.55052E+01]),
        nist_form_factor([7.902609E+00  1.01091E+01; 8.447890E+00  1.00992E+01]),
    ]
)
atomdata = innerjoin(atomcoords, form_factors, on = :atom)

Row,atom,x,y,z,f
,String,Float64,Float64,Float64,Float64
1,Y3+,0.75,0.75,0.75,35.5803
2,Y3+,1.03241,1.0,0.75,35.5803
3,O2-,0.6092,0.8482,0.6197,10.1065


In [98]:
for (atom0, x0, y0, z0, f0) in eachrow(atomdata)
    for xyz in unique(map(op -> (op * [x0, y0, z0, 1]), sg))
        xyz = map(xyz) do t
            round(t - round(t, RoundDown), digits = 4)
        end
        if isempty(subset(atomdata, [:x, :y, :z] => ByRow((x, y, z) -> all((x, y, z) .== xyz))))
            push!(atomdata, (atom0, xyz..., f0))
        end
    end
end
atomdata

Row,atom,x,y,z,f
,String,Float64,Float64,Float64,Float64
1,Y3+,0.75,0.75,0.75,35.5803
2,Y3+,1.03241,1.0,0.75,35.5803
3,O2-,0.6092,0.8482,0.6197,10.1065
4,Y3+,0.75,0.25,0.25,35.5803
5,Y3+,0.25,0.25,0.75,35.5803
6,Y3+,0.25,0.75,0.25,35.5803
7,Y3+,0.25,0.25,0.25,35.5803
8,Y3+,0.25,0.75,0.75,35.5803
9,Y3+,0.75,0.75,0.25,35.5803


In [99]:
structure_amplitude(h, k, l; atomdata = atomdata) = sum(eachrow(atomdata)) do row
    _, x, y, z, f = collect(row)
    f * cispi(2 * (h * x + k * y + l * z))
end

structure_amplitude (generic function with 1 method)

In [100]:
refs = DataFrame(h = Int[], k = Int[], l = Int[])
for h in -20:20, k in -20:20, l in -20:20
    push!(refs, (h, k, l))
end
refs

Row,h,k,l
,Int64,Int64,Int64
1,-20,-20,-20
2,-20,-20,-19
3,-20,-20,-18
4,-20,-20,-17
5,-20,-20,-16
6,-20,-20,-15
7,-20,-20,-14
8,-20,-20,-13
9,-20,-20,-12


In [101]:
a_inv = ustrip(u"keV", 10.594u"Å", Spectral())
bragg_tth(h, k, l; a = 1 / a_inv, λ = 1 / E) = try round(rad2deg(2asin(hypot(h, k, l) * λ / 2a)), digits = 4)
catch
    180.0
end
tth_ax = RotAxis(0, 0, 1)
omega_ax = RotAxis(0, 0, 1)
phi_ax = RotAxis(0, 0, -1)
chi_ax = RotAxis(-1, 0, 0)
χ = 54.7112
phi_ax = chi_ax(deg2rad(χ)) * phi_ax
UB = p4p["ORT"] * ustrip(uconvert(u"keV", 1u"Å^-1", Spectral()))
# UB *= p4p["CELL"][1] / 10.6046
UB = chi_ax(deg2rad(χ)) * UB
function equator_angles(h, k, l, tth)
    q = UB * [h, k, l]
    eq = [cosd(90 + tth/2), sind(90 + tth/2), 0]
    try map(orient_angles(phi_ax, omega_ax, q => eq)) do angles
            @.rem(rad2deg(angles), 360, RoundDown)
        end |> Tuple
    catch
        (nothing, nothing)
    end
end
DataFrames.transform!(refs, [:h, :k, :l] => ByRow(bragg_tth) => [:tth])
DataFrames.transform!(refs, [:h, :k, :l] => ByRow(abs ∘ structure_amplitude) => [:F])
DataFrames.subset!(refs, :tth => tth -> @.tth != 180.0 && tth != 0.0);

In [102]:
angles_raw = DataFrame(h = Int[], k = Int[], l = Int[], tth = Float64[], F = Float64[], phi = Float64[], omega = Float64[])
for (h, k, l, tth0, FF) in eachrow(refs),
    tth in (tth0, -tth0),
    ang in equator_angles(h, k, l, tth)
    if !isnothing(ang)
        push!(angles_raw, (h, k, l, tth, FF, round(ang[1], digits = 4), round(ang[2], digits = 4)))
    end
end

In [103]:
function omega_range(distance)
    data = [
         39.033 -172.50 -7.50
         48.890 -183.19  3.19
         59.034 -191.94 11.94
         69.087 -199.38 19.38
         96.319 -214.94 34.94
        109.357 -220.31 40.38
        128.820 -226.88 46.88
        138.855 -229.62 49.62
        159.357 -234.44 54.44
        178.843 -252.12 72.12
        209.205 -270.00 90.00
    ]
    min = linear_interpolation(data[:, 1], data[:, 2])
    max = linear_interpolation(data[:, 1], data[:, 3])
    min(distance), max(distance)
end

omega_range (generic function with 1 method)

In [104]:
function collision_check(tth, omega, distance = 96.319, tthDneg = 80, scan = 4)
    tthD = tth < 0 ? -tthDneg : 2tth - tthDneg
    omega_1, omega_2 = omega_range(distance) .+ tthD
    omega_mean = (omega_1 + omega_2) / 2
    omega_gap = abs(omega_1 - omega_2) / 2
    abs(rem(omega + scan / 2 - omega_mean, 360, RoundNearest)) < omega_gap &&
    abs(rem(omega - scan / 2 - omega_mean, 360, RoundNearest)) < omega_gap &&
    90 < abs(tth) < 110
end

collision_check (generic function with 4 methods)

In [105]:
angles = subset(angles_raw, [:tth, :omega] => ByRow(collision_check))

Row,h,k,l,tth,F,phi,omega
,Int64,Int64,Int64,Float64,Float64,Float64,Float64
1,-11,-2,-1,-109.407,225.784,55.5709,144.016
2,-11,-2,-1,-109.407,225.784,354.78,106.576
3,-11,-2,0,-108.766,45.4252,68.1314,154.62
4,-11,-2,0,-108.766,45.4252,340.49,96.6134
5,-11,-2,1,-109.407,163.66,76.4483,162.629
6,-11,-2,1,-109.407,163.66,330.732,87.9636
7,-11,-1,-1,-107.491,45.082,46.059,143.957
8,-11,-1,-1,-107.491,45.082,348.217,108.552
9,-11,-1,0,-106.857,40.2575,60.1556,155.01


In [106]:
bond = innerjoin(angles[angles.tth .> 0, :], DataFrames.transform(angles, :tth => -, renamecols = false),
    on = [:tth, :phi, :h, :k, :l, :F],
    renamecols = "_pos" => "_neg",
)

Row,h,k,l,tth,F,phi,omega_pos,omega_neg
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,-10,-4,2,105.595,535.453,98.918,280.819,175.224
2,-10,-4,3,108.766,26.199,102.599,289.0,180.233
3,-10,-3,1,99.3871,284.266,86.5213,269.011,169.624
4,-10,-3,2,101.234,26.8971,92.0509,277.533,176.299
5,-10,-3,3,104.341,323.731,96.4519,286.019,181.678
6,-10,-3,4,108.766,41.5828,100.084,294.589,185.823
7,-10,-2,1,96.3289,27.7896,78.579,266.404,170.075
8,-10,-2,2,98.1611,682.842,85.0616,275.262,177.101
9,-10,-2,3,101.234,42.5395,90.1754,283.987,182.753


In [107]:
function hklpos(h, k, l)
    h != 0 ? h > 0 :
    k != 0 ? k > 0 :
    l > 0
end
function dphi!(df::DataFrame)
    transform!(df, r"phi" => ByRow((p...) -> abs(rem(maximum(p) - minimum(p), 360, RoundNearest))) => :dphi)
end

dphi! (generic function with 1 method)

In [108]:
frid = let data = angles
    df = innerjoin(subset(angles, [:h, :k, :l] => ByRow(hklpos)), DataFrames.transform(angles, [:h, :k, :l] .=> -, renamecols = false),
        on = [:tth, :h, :k, :l, :F],
        renamecols = "" => "_frid"
    )
    dphi!(df)
    subset(df, :dphi => ByRow(p -> p < 10))
end

Row,h,k,l,tth,F,phi,omega,phi_frid,omega_frid,dphi
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,11,2,1,-109.407,225.784,354.78,286.576,354.78,106.576,0.0
2,11,2,0,-108.766,45.4252,340.49,276.613,340.49,96.6134,0.0
3,11,2,-1,-109.407,163.66,330.732,267.964,330.732,87.9636,0.0
4,11,1,1,-107.491,45.082,348.217,288.552,348.217,108.552,0.0
5,11,1,0,-106.857,40.2575,333.852,278.132,333.852,98.1324,0.0
6,11,1,-1,-107.491,32.3868,324.387,269.273,324.387,89.2734,0.0
7,11,1,-2,-109.407,41.3678,317.252,260.799,317.252,80.799,0.0
8,11,0,1,-106.857,39.0855,339.443,288.485,339.443,108.485,0.0
9,11,0,0,-106.225,31.0648,326.069,278.167,326.069,98.1666,0.0


In [109]:
frid_ideal = innerjoin(subset(angles, [:h, :k, :l] => ByRow(hklpos)), DataFrames.transform(angles, [:h, :k, :l] .=> -, renamecols = false),
    on = [:tth, :phi, :h, :k, :l, :F],
    renamecols = "" => "_frid"
)

Row,h,k,l,tth,F,phi,omega,omega_frid
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,11,2,1,-109.407,225.784,354.78,286.576,106.576
2,11,2,0,-108.766,45.4252,340.49,276.613,96.6134
3,11,2,-1,-109.407,163.66,330.732,267.964,87.9636
4,11,1,1,-107.491,45.082,348.217,288.552,108.552
5,11,1,0,-106.857,40.2575,333.852,278.132,98.1324
6,11,1,-1,-107.491,32.3868,324.387,269.273,89.2734
7,11,1,-2,-109.407,41.3678,317.252,260.799,80.799
8,11,0,1,-106.857,39.0855,339.443,288.485,108.485
9,11,0,0,-106.225,31.0648,326.069,278.167,98.1666


In [110]:
eccentr = let data = frid
    df = innerjoin(data[data.tth .> 0, :], DataFrames.transform(data, :tth => -, renamecols = false),
        on = [:tth],
        renamecols = "_pos" => "_neg"
    )
    dphi!(df)
    subset(df, :dphi => ByRow(p -> p < 10))
end

Row,h_pos,k_pos,l_pos,tth,F_pos,phi_pos,omega_pos,phi_frid_pos,omega_frid_pos,dphi_pos,h_neg,k_neg,l_neg,F_neg,phi_neg,omega_neg,phi_frid_neg,omega_frid_neg,dphi_neg,dphi
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10,1,-5,109.407,373.703,92.6096,120.675,92.6096,300.675,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
2,10,1,-5,109.407,373.703,301.496,348.733,301.496,168.733,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
3,10,-1,-5,109.407,372.315,82.2804,121.408,82.2804,301.408,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
4,10,-1,-5,109.407,372.315,290.218,348.0,290.218,168.0,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
5,9,6,-3,109.407,298.339,118.137,114.017,118.137,294.017,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
6,9,3,-6,109.407,228.827,108.902,129.348,108.902,309.348,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
7,9,3,-6,109.407,228.827,306.931,340.059,306.931,160.059,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
8,9,-3,-6,109.407,212.226,78.5283,131.427,78.5283,311.427,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198
9,9,-3,-6,109.407,212.226,274.053,337.98,274.053,157.98,0.0,11,2,1,225.784,354.78,286.576,354.78,106.576,0.0,5.2198


In [111]:
eccentr_ideal = let data = frid_ideal
    df = innerjoin(data[data.tth .> 0, :], DataFrames.transform(data, :tth => -, renamecols = false),
        on = [:tth],
        renamecols = "_pos" => "_neg"
    )
    dphi!(df)
    subset(df, :dphi => ByRow(p -> p < 10))
end

Row,h_pos,k_pos,l_pos,tth,F_pos,phi_pos,omega_pos,omega_frid_pos,h_neg,k_neg,l_neg,F_neg,phi_neg,omega_neg,omega_frid_neg,dphi
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,6,9,-3,109.407,246.341,347.238,341.773,161.773,11,2,1,225.784,354.78,286.576,106.576,7.5418
2,5,10,-1,109.407,325.073,2.6497,346.619,166.619,11,2,1,225.784,354.78,286.576,106.576,7.8695
3,1,11,-2,109.407,225.235,1.4593,324.165,144.165,11,2,1,225.784,354.78,286.576,106.576,6.6791
4,1,-11,2,109.407,231.674,3.3746,135.168,315.168,11,2,1,225.784,354.78,286.576,106.576,8.5944
5,6,8,-5,108.766,44.0924,332.902,332.214,152.214,11,2,0,45.4252,340.49,276.613,96.6134,7.588
6,3,10,-4,108.766,35.9721,346.017,323.752,143.752,11,2,0,45.4252,340.49,276.613,96.6134,5.5269
7,0,10,-5,108.766,42.9973,342.165,305.561,125.561,11,2,0,45.4252,340.49,276.613,96.6134,1.6744
8,3,9,-6,109.407,297.451,331.964,314.985,134.985,11,2,-1,163.66,330.732,267.964,87.9636,1.2323
9,7,8,-3,106.857,247.93,343.822,345.252,165.252,11,1,0,40.2575,333.852,278.132,98.1324,9.9697


In [112]:
eccentr_perfect = innerjoin(frid_ideal[frid_ideal.tth .> 0, :], DataFrames.transform(frid_ideal, :tth => -, renamecols = false),
    on = [:tth, :phi],
    renamecols = "_pos" => "_neg"
)

Row,h_pos,k_pos,l_pos,tth,F_pos,phi,omega_pos,omega_frid_pos,h_neg,k_neg,l_neg,F_neg,omega_neg,omega_frid_neg
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64


In [113]:
XLSX.openxlsx("YEu_AM-17-22_suppose.xlsx", mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, "angles")
    XLSX.writetable!(xf[1], angles)
    sheet = XLSX.addsheet!(xf, "bond")
    XLSX.writetable!(sheet, bond)
    sheet = XLSX.addsheet!(xf, "frid")
    XLSX.writetable!(sheet, frid)
    sheet = XLSX.addsheet!(xf, "frid_ideal")
    XLSX.writetable!(sheet, frid_ideal)
    sheet = XLSX.addsheet!(xf, "eccentr")
    XLSX.writetable!(sheet, eccentr)
    sheet = XLSX.addsheet!(xf, "eccentr_ideal")
    XLSX.writetable!(sheet, eccentr_ideal)
    sheet = XLSX.addsheet!(xf, "eccentr_perfect")
    XLSX.writetable!(sheet, eccentr_perfect)
end

In [ ]:
function peak_coords((h, k, l), (tth, phi, omega);
        distance = 96.319,
        center = (387, 505),
        px = 0.1353,
        detector = Detector([0, -px, 0], [0, 0, px], [distance, center[1] * px, -center[2] * px]),
)
    tth, omega, phi = deg2rad.((tth, omega, phi))
    k = [E, 0, 0] + omega_ax(omega) * phi_ax(phi) * UB * [h, k, l]
    @show hypot(k...) - E
    detector = tth_ax(tth) * detector
    intersect_coord(detector, XRay(k))
end

In [ ]:
peak_coords((2, 4, -10), (-80, 168.6-0.47, 70.84+180)) + Vec2d(0, 4.9)